In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

"""
📊 **Plot SNR Time-Series for MRI Metabolism Study**
- Extracts SNR values from **Excel files** and plots time-series trends.
- Converts scan timepoints into **actual hours post-injection**.
- Saves the **final plot** inside the `snr_plots/` folder.
"""

def plot_snr_time_series(excel_file, output_folder, probe):
    """
    📌 **Generates and saves SNR plots for 3FDG or 3FDGal.**

    ✅ **Input Parameters**:
    - `excel_file`: Path to the **Excel file** containing SNR values.
    - `output_folder`: Directory where the **plot image** will be saved.
    - `probe`: **"p1"** for 3FDG or **"p2"** for 3FDGal.

    ✅ **Output**:
    - Saves a **high-resolution plot** of SNR changes over time.
    - Displays the plot in a well-formatted layout.
    """

    os.makedirs(output_folder, exist_ok=True)  # Ensure output folder exists

    # 🔍 **Load the Excel file**
    try:
        df = pd.read_excel(excel_file, sheet_name="Summary")  # Load SNR Summary Sheet
        print(f"✅ Successfully loaded data from: {excel_file}")
    except Exception as e:
        print(f"❌ Error loading the Excel file: {e}")
        return

    # 🏷 **Extract Timepoints & Metabolites from Filenames**
    df["Timepoint"] = df["Filename"].str.extract(r"t(\d+)").astype(float)
    df["Metabolite"] = df["Filename"].str.extract(r"m(\d+)")

    # 🎯 **Map Metabolite Names Based on Selected Probe**
    metabolite_names = {
        "p1": {"1": "3FDGlucose", "3": "3FDGluconic Acid", "4": "3FDSorbitol"},
        "p2": {"1": "3FDGal", "3": "Galactonic Acid", "4": "Galactitol"}
    }
    df["Metabolite"] = df["Metabolite"].map(metabolite_names[probe])

    # ⏳ **Convert Timepoints to Actual Hours Post-Injection (Corrected)**
    time_mapping = {1: 1, 2: 2, 3: 3}  # T1 → 1h, T2 → 2h, T3 → 3h
    df["Time (hours post-injection)"] = df["Timepoint"].map(time_mapping)

    # 🎨 **Define Colors for Each Metabolite (Matching Previous Scripts)**
    color_mapping = {
        "p1": {"3FDGlucose": "limegreen", "3FDGluconic Acid": "red", "3FDSorbitol": "yellow"},
        "p2": {"3FDGal": "cyan", "Galactonic Acid": "magenta", "Galactitol": "gold"}
    }

    # 📊 **Plot Configuration**
    plt.figure(figsize=(10, 6))

    for metabolite, color in color_mapping[probe].items():
        subset = df[df["Metabolite"] == metabolite]
        plt.plot(
            subset["Time (hours post-injection)"],
            subset["SNR"],
            marker="o",
            markersize=8,
            linewidth=2,
            color=color,
            label=metabolite
        )

    # 📌 **Style Adjustments**
    plt.xlabel("Time (hours post-injection)", fontsize=12)
    plt.ylabel("SNR", fontsize=12)
    plt.title(f"SNR Over Time - {probe.upper()} Probe", fontsize=14, fontweight="bold")
    plt.legend(fontsize=10, title="Metabolites")
    plt.xticks([1, 2, 3])
    plt.grid(True, linestyle="--", alpha=0.7)

    # 📂 **Save the Plot**
    output_path = os.path.join(output_folder, f"SNR_Plot_{probe}.png")
    plt.savefig(output_path, dpi=300, bbox_inches="tight")
    print(f"✅ Plot saved at: {output_path}")

    # 📊 **Display the Plot**
    plt.show()


# 🔹 **User Input: Define Data Paths**
repo_directory = r"YOUR_LOCAL_PATH_HERE/FMRI-Metabolism-Quantification-main"

# 🎯 **Choose probe ('p1' for 3FDG or 'p2' for 3FDGal)**
probe = "p2"  # Change to "p1" for 3FDG

# 📂 **Define Excel Data Folder**
snr_directory = os.path.join(repo_directory, "snr_results")  # Ensure correct folder structure

# 🔍 **Find the Latest Excel File**
excel_files = [f for f in os.listdir(snr_directory) if f.endswith(".xlsx")]

if not excel_files:
    print("⚠ No Excel files found in the directory!")
else:
    excel_file = os.path.join(snr_directory, excel_files[0])  # Use the first Excel file found
    print(f"✅ Using Excel file: {excel_file}")

    # 📂 **Run the Function to Generate & Save the Plot**
    output_folder = os.path.join(repo_directory, "snr_plots")
    plot_snr_time_series(excel_file, output_folder, probe)
